**2016.12.10 - work log - prelim_month - single name match error**

 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="https://research.local:8000/user/jonathanmorgan/notebooks/work/sourcenet/django/research/work/msu_phd_work/2016.12.10-work_log-prelim_month-single_name_match_error.ipynb#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="https://research.local:8000/user/jonathanmorgan/notebooks/work/sourcenet/django/research/work/msu_phd_work/2016.12.10-work_log-prelim_month-single_name_match_error.ipynb#Setup---Imports" data-toc-modified-id="Setup---Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="https://research.local:8000/user/jonathanmorgan/notebooks/work/sourcenet/django/research/work/msu_phd_work/2016.12.10-work_log-prelim_month-single_name_match_error.ipynb#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li></ul></li><li><span><a href="https://research.local:8000/user/jonathanmorgan/notebooks/work/sourcenet/django/research/work/msu_phd_work/2016.12.10-work_log-prelim_month-single_name_match_error.ipynb#Coding-to-look-into" data-toc-modified-id="Coding-to-look-into-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Coding to look into</a></span><ul class="toc-item"><li><span><a href="https://research.local:8000/user/jonathanmorgan/notebooks/work/sourcenet/django/research/work/msu_phd_work/2016.12.10-work_log-prelim_month-single_name_match_error.ipynb#Match-for-just-first-name?---DONE" data-toc-modified-id="Match-for-just-first-name?---DONE-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Match for just first name? - DONE</a></span></li></ul></li></ul></div>

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
import datetime

print( "packages imported at " + str( datetime.datetime.now() ) )

packages imported at 2016-12-11 21:49:53.417627


In [2]:
%pwd

'/home/jonathanmorgan/work/sourcenet/django/dev/work/msu_phd_work'

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [3]:
%run django_init.py

django initialized at 2016-12-11 21:49:58.222090


# Coding to look into

- Back to [Table of Contents](#Table-of-Contents)

Coding decisions to look at more closely:

## Match for just first name? - DONE

- Back to [Table of Contents](#Table-of-Contents)

First name "Kate" was matched to "Kate Gosselin" but "Gosselin" is nowhere in the article.

- Article Data 2980, article 20739 - 11003 (AS) - Gosselin, Kate ( id = 1608; capture_method = OpenCalais_REST_API_v2 ) (mentioned; individual) ==> name: Kate
    
    - Not sure where "Gosselin" came from - need to look into the lookup for "Kate".
    - article 20739 - https://data.jrn.cas.msu.edu/sourcenet-dev/sourcenet/article/article_data/view_with_text/?article_id=20739
    - article data 2980 - https://data.jrn.cas.msu.edu/sourcenet-dev/sourcenet/article/article_data/view/?article_id=20739&article_data_id_select=2980
    


In [4]:
# imports
from sourcenet.article_coding.manual_coding.manual_article_coder import ManualArticleCoder
from sourcenet.models import Article_Subject

# declare variables
my_coder = None
subject = None
person_name = ""
person_instance = None
person_match_list = None

# create ManualArticleCoder and Article_Subject instance
my_coder = ManualArticleCoder()
subject = Article_Subject()

# set up look up of "Kate"
person_name = "Kate"

# lookup person - returns person and confidence score inside
#    Article_Person descendent instance.
subject = my_coder.lookup_person( subject, 
                                  person_name,
                                  create_if_no_match_IN = False,
                                  update_person_IN = False )

# retrieve information from Article_Person
person_instance = subject.person
person_match_list = subject.person_match_list  # list of Person instances

if ( person_instance is not None ):

    # Found person for "Kate":
    print( "Found person for \"" + str( person_name ) + "\": " + str( person_instance ) )
    
else:
    
    # no person instance found.
    print( "No person instance found for \"" + str( person_name ) + "\"" )
    
#-- END check to see if person_instance --#

if ( ( person_match_list is not None ) and ( len( person_match_list ) > 0 ) ):

    print( "match list:" )
    for match_person in person_match_list:
        
        # output each person for now.
        print( "- " + str( match_person ) )
        
    #-- END loop over person_match_list --#

else:
    
    print( "match list is None or empty." )

#-- END check to see if there is a match list.

No person instance found for "Kate"
match list is None or empty.


_Update - Well, it no longer returns anything for "Kate".  Need to run unit tests._

For more precise detail, looked at the underlying way the lookup method finds a person for a name:

In [5]:
# imports
from sourcenet.models import Person

# declare variables
person_name = ""
person_instance = None
lookup_status = ""

# lookup Person - this is the way it works down in Article_Coder.lookup_person().
person_name = "Kate"
person_instance = Person.get_person_for_name( person_name, create_if_no_match_IN = False )
lookup_status = Person.get_person_lookup_status( person_instance )

# what we got?
print( "Person instance: " + str( person_instance ) + "; lookup_status: " + str( lookup_status ) )

# String lookup?
person_instance = Person.get_person_for_name( person_name,
                                              create_if_no_match_IN = False,
                                              do_strict_match_IN = True,
                                              do_partial_match_IN = False )

# what we got?
print( "Person instance strict lookup: " + str( person_instance ) )



Person instance: 1608 - Gosselin, Kate ( Zondervan ); lookup_status: found
Person instance strict lookup: None


Is there only one person with first name Kate?

In [6]:
# imports
from sourcenet.models import Person

# declare variables
name_string = ""
test_person_qs = None
test_person = None
test_person_count = -1

# do a lookup, filtering on first name of "Kate".
name_string = "Kate"
test_person_qs = Person.objects.filter( first_name = name_string )

# got anything at all?
if ( test_person_qs is not None ):

    # process results - count...
    test_person_count = test_person_qs.count()
    print( "Found " + str( test_person_count ) + " matches:" )

    # ...and loop.
    for test_person in test_person_qs:

        # output person
        print( "- " + str( test_person ) )
        
    #-- END loop over matching persons. --#
    
#-- END check to see if None --#

Found 1 matches:
- 1608 - Gosselin, Kate ( Zondervan )


So... If there is a single match in the database for a single name part (first name or last name), but the match contains more than just the first name, I don't want to call that a match unless there is some sort of associated ID that also matches.

Make sure that single names are always considered the first name:

In [7]:
# functions
def output_person_name_details( person_IN ):

    if ( person_IN is not None ):

        print( "Person " + str( person_IN ) + ":" )
        print( "- first_name: " + str( person_IN.first_name ) )
        print( "- middle_name: " + str( person_IN.middle_name ) )
        print( "- last_name: " + str( person_IN.last_name ) )
        print( "- name_prefix: " + str( person_IN.name_prefix ) )
        print( "- name_suffix: " + str( person_IN.name_suffix ) )
        print( "- nickname: " + str( person_IN.nickname ) )
        print( "- full_name_string: " + str( person_IN.full_name_string ) )
        print( "- original_name_string: " + str( person_IN.original_name_string ) )
    
    else:
        
        print( "Can't output details since nothing (None) passed in." )
        
    #-- END Check to see if Person is None --# 
    
#-- END function output_person_name_details() --#
    
# imports
from sourcenet.models import Person

# declare variables
person_instance = None
name_string = ""
parsed_name = None

# see if HumanName consistently places single name in first name.
person_instance = Person()
name_string = "Smith"
person_instance.set_name( name_string )
output_person_name_details( person_instance )

# see if HumanName consistently places single name in first name.
person_instance = Person()
name_string = "Kevin"
person_instance.set_name( name_string )
output_person_name_details( person_instance )

# see if HumanName consistently places single name in first name.
person_instance = Person()
name_string = "Huxtable"
person_instance.set_name( name_string )
output_person_name_details( person_instance )

# see if HumanName consistently places single name in first name.
person_instance = Person()
name_string = "Kate"
person_instance.set_name( name_string )
output_person_name_details( person_instance )

# see if HumanName consistently places single name in first name.
person_instance = Person()
name_string = "Von Williams"
person_instance.set_name( name_string )
output_person_name_details( person_instance )

Person None, Smith:
- first_name: Smith
- middle_name: None
- last_name: None
- name_prefix: None
- name_suffix: None
- nickname: None
- full_name_string: Smith
- original_name_string: Smith
Person None, Kevin:
- first_name: Kevin
- middle_name: None
- last_name: None
- name_prefix: None
- name_suffix: None
- nickname: None
- full_name_string: Kevin
- original_name_string: Kevin
Person None, Huxtable:
- first_name: Huxtable
- middle_name: None
- last_name: None
- name_prefix: None
- name_suffix: None
- nickname: None
- full_name_string: Huxtable
- original_name_string: Huxtable
Person None, Kate:
- first_name: Kate
- middle_name: None
- last_name: None
- name_prefix: None
- name_suffix: None
- nickname: None
- full_name_string: Kate
- original_name_string: Kate
Person Williams, Von:
- first_name: Von
- middle_name: None
- last_name: Williams
- name_prefix: None
- name_suffix: None
- nickname: None
- full_name_string: Von Williams
- original_name_string: Von Williams


Sure looks like single name part always goes into first name.  So, if I do a straight up lookup, and the original name parses out to only have a first name, and the result has more than a first name, then we call it not a match, store the match as an alternate.

Test out the check to see if there is more than a first name.

In [ ]:
# imports
from sourcenet.models import Person

# declare variables
person_name = ""
person_instance = None
is_just_first_name = False

# is single name part?
person_name = "Kate"
is_just_first_name = Person.is_single_name_part( person_name )
print( str( person_name ) + " - " + str( is_just_first_name ) )

# is single name part?
person_name = "Kate Smith"
is_just_first_name = Person.is_single_name_part( person_name )
print( str( person_name ) + " - " + str( is_just_first_name ) )

# is single name part?
person_name = "Michael W. Smith"
is_just_first_name = Person.is_single_name_part( person_name )
print( str( person_name ) + " - " + str( is_just_first_name ) )

# is single name part?
person_name = "Michael  "
is_just_first_name = Person.is_single_name_part( person_name )
print( str( person_name ) + " - " + str( is_just_first_name ) )

# is single name part?
person_name = "Mr. Michael"
is_just_first_name = Person.is_single_name_part( person_name )
print( str( person_name ) + " - " + str( is_just_first_name ) )

Here is the fix I came up with (article_coder.py lines 1308 to 1365):

In [ ]:
# 1307...
                    # check to see if the name passed in is a single name.
                    is_single_name_part = Person.is_single_name_part( full_name_IN )
                    if ( is_single_name_part == True ):

                        # yes.  Do strict lookup with no partial match to see if
                        #     this is an exact match.
                        temp_person = Person.get_person_for_name( full_name_IN,
                                                                  create_if_no_match_IN = False,
                                                                  do_strict_match_IN = True,
                                                                  do_partial_match_IN = False )
                        temp_lookup_status = Person.get_person_lookup_status( temp_person )
                        
                        # exact match?
                        if ( ( temp_person is None ) or ( person_instance.id != temp_person.id ) ):
                        
                            # we have a single word name, lookup and strict
                            #     lookup result in different matches.
                                                        
                            # further verify by checking if just one match for
                            #     the name passed in and first_name, ignoring
                            #     other name fields.
                            test_person_qs = Person.objects.filter( first_name__iexact = full_name_IN )
                            test_person_count = test_person_qs.count()
                            if ( test_person_count == 1 ):

                                # This is a relatively rare scenario - a single
                                #     name part matches to the only person in
                                #     the database that contains that name part
                                #     in their first name.  For our purposes,
                                #     this is not a match.  Make a new person
                                #     for the single name part, set match status
                                #     to None.
                                match_status = self.MATCH_STATUS_NONE
                                person_instance = Person.create_person_for_name( full_name_IN )
                                self.output_debug( "In " + me + ": name " + str( full_name_IN ) + " is first name of one person ( " + str( person_instance ) + " ) who has more name information.  This is not a reliable match, so creating new Person with just name passed in." )
                                
                            elif ( test_person_count > 1 ):
                            
                                # make list of IDs of multiple matches.
                                multiple_list = []
                                for test_person in test_person_qs:
                                    
                                    # add ID of each person to list.
                                    multiple_list.append( test_person )
                                    
                                #-- END loop over multiple matches. --#
                                
                                self.output_debug( "In " + me + ": name " + str( full_name_IN ) + " is first name of more than one person ( " + str( multiple_list ) + " ) who just has that first name.  But we found an exact match.  This makes no sense." )
                                
                            else:
                            
                                self.output_debug( "In " + me + ": name " + str( full_name_IN ) + " is not first name of any person, and yet it was matched to person: " + str( person_instance ) + ".  This makes no sense." )                                

                            #-- END check to see if one person with same first name --#
                                
                        #-- END check to see if our match is an exact match --#
                    
                    #-- END check to see if single name part --#
# ...1365

Now, to run unit tests.  Command is:

    python manage.py test sourcenet.tests
    
must be in Django project folder and have activated your virtualenv.

Tests passed.  Plus little tests here.  We'll call that good for now, moving back to assessing each single-word name.